# **Multi Server MCP Client**

## 1. **Common transport**
### 단일 Session

In [11]:
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    pprint.pprint(client.get_tools())

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13896b9c0>),
 StructuredTool(name='get_weather_korea', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_koreaArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x138d349a0>)]


### 유지 Session

In [12]:
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

# 1. Create a client
client =  MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. Initialize a model
await client.__aenter__()

# 3. Load Tools
tools = client.get_tools()
pprint.pprint(tools)

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13896bf60>),
 StructuredTool(name='get_weather_korea', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_koreaArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13896ab60>)]


## 2. SSE 통신
Model Context Protocol을 Server Sent Events를 통해 구현한 프로토콜. HTTP를 통해 원격서비스에 연결할 수 있다.
- MCP 서버에서 먼저 running되고 있어야 한다.

In [13]:
# 단일 세션
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient

async with MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    pprint.pprint(client.get_tools())

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x138d34540>),
 StructuredTool(name='get_weather_korea', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weather_koreaArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x138d349a0>)]


In [24]:
# 유지 세션
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from openai import AsyncOpenAI

# OpenAI 클라이언트 생성
client_openai = AsyncOpenAI()

# OpenAI 모델을 langchain 형식으로 래핑
async def openai_chat(messages):
    response = await client_openai.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "system", "content": prompt}] + [{"role": "user", "content": messages}],
        temperature=0
    )
    return response.choices[0].message.content

# 1. Create a client
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. Initialize a model
await client.__aenter__()

# 3. Load Tools
tools = client.get_tools()

# 4. Define Prompt
prompt = (
    'You are a smart agent.'
    'Use `weather` tool to get the weather in a given city.'
    'Answer in Korean.'
)

# 5. agent
agent = create_react_agent(openai_chat, tools, prompt=prompt)

agent_response = await agent.ainvoke({"messages": "What is the weather in Seoul?"})
agent_response

AttributeError: 'function' object has no attribute 'bind_tools'

## 3. Std I/O 통신
로컬 프로세스에 적합, 통신을 위한 표준 입출력

In [23]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

server_params = StdioServerParameters(
    command="/Users/choejaehun/Desktop/mcp_tutorial/.venv/bin/python",
    args=["mcp_server_local.py"],
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialization
        await session.initialize()
        
        # Get Tools
        tools = await load_mcp_tools(session)
        pprint.pprint(tools)
        
        # Create Agent
        agent = create_react_agent(models, tools, prompt=prompt)
        agent_response = await agent.ainvoke({"messages": "What is the weather in Seoul?"})
        pprint.pprint(agent_response)

[StructuredTool(name='get_weather', description='Get the weather in a given city', args_schema={'properties': {'city': {'title': 'City', 'type': 'string'}}, 'required': ['city'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x13a9351c0>)]
{'messages': [HumanMessage(content='What is the weather in Seoul?', additional_kwargs={}, response_metadata={}, id='b02a8455-1ea3-49f7-aa86-047259c8e572'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_m3uJPeRL5BneNbwc4IbUUIzs', 'function': {'arguments': '{"city":"Seoul"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 75, 'total_tokens': 91, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_detail

In [33]:
# 유지 세션
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient
from openai import AsyncOpenAI
import json

# 1. Create a client
client = MultiServerMCPClient(
    {
        "information": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. Initialize a model
await client.__aenter__()

# 3. Load Tools
tools = client.get_tools()

# 4. Define system prompt
system_prompt = (
    'You are a smart agent.'
    'Use `information` tool to get the color or name in a given target.'
    'Answer in Korean.'
)

# 5. Create OpenAI client
openai_client = AsyncOpenAI()

# 6. Define function calling format
function_calls = [
    {
        "name": tool.name,
        "description": tool.description,
        "parameters": tool.args_schema
    }
    for tool in tools
]

QUESTION = "What is the name of the apple?"

# 7. Make API call
response = await openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": QUESTION}
    ],
    functions=function_calls,
    function_call="auto"
)

# 8. Process the response
# 이 코드는 OpenAI API의 응답을 처리하는 부분입니다.
# OpenAI의 응답에서 첫 번째 메시지를 가져옵니다
message = response.choices[0].message
print(message)

# function call이 있는 경우 (날씨 도구를 사용하라는 응답인 경우)
if message.function_call:
    # function call의 이름과 인자를 추출합니다
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)
    
    # 요청된 도구를 찾아서 실행합니다
    for tool in tools:
        if tool.name == function_name:
            # 도구를 비동기로 실행하고 결과를 받습니다
            tool_response = await tool.ainvoke(function_args)
            print(f"Tool response: {function_name}")
            print(f"Tool response: {tool_response}")
            
            # 도구의 결과를 포함하여 최종 응답을 생성합니다
            final_response = await openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": QUESTION},
                    {"role": "assistant", "content": None, "function_call": {"name": function_name, "arguments": message.function_call.arguments}},
                    {"role": "function", "name": function_name, "content": tool_response}
                ]
            )
            print(f"Final response: {final_response.choices[0].message.content}")
else:
    # function call이 없는 경우 직접 응답을 출력합니다
    print(f"Response: {message.content}")

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "target": "apple"\n}', name='get_name'), tool_calls=None)
Tool response: get_name
Tool response: apple's name is ASH
Final response: 사과의 이름은 ASH입니다.


In [39]:
import pprint
from langchain_mcp_adapters.client import MultiServerMCPClient
from openai import AsyncOpenAI
import json

# 1. Create a client
client = MultiServerMCPClient(
    {
        "information": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)

# 2. Initialize a model
await client.__aenter__()

# 3. Load Tools
tools = client.get_tools()

# 4. Define system prompt
system_prompt = (
    'You are a smart agent.'
    'Use `information` tool to get the color or name in a given target.'
    'Answer in Korean.'
)

# 5. Create OpenAI client
openai_client = AsyncOpenAI()

# 6. Define function calling format
function_calls = [
    {
        "name": tool.name,
        "description": tool.description,
        "parameters": tool.args_schema
    }
    for tool in tools
]

Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


In [40]:
QUESTION = "What is the color of the apple?"
QUESTION1 = "What is the name of the apple?"

In [41]:
# 7. Make API call
response = await openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": QUESTION}
    ],
    functions=function_calls,
    function_call="auto"
)

# 8. Process the response
# 이 코드는 OpenAI API의 응답을 처리하는 부분입니다.
# OpenAI의 응답에서 첫 번째 메시지를 가져옵니다
message = response.choices[0].message
pprint.pprint(response)
pprint.pprint(message)
# 8. Process the response
# 이 코드는 OpenAI API의 응답을 처리하는 부분입니다.
# OpenAI의 응답에서 첫 번째 메시지를 가져옵니다
message = response.choices[0].message
pprint.pprint(response)
pprint.pprint(message)

if message.function_call:
    # function call의 이름과 인자를 추출합니다
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)
    
    # 요청된 도구를 찾아서 실행합니다
    for tool in tools:
        if tool.name == function_name:
            # 도구를 비동기로 실행하고 결과를 받습니다
            tool_response = await tool.ainvoke(function_args)
            print(f"Tool response: {function_name}")
            print(f"Tool response: {tool_response}")
            
            # 도구의 결과를 포함하여 최종 응답을 생성합니다
            final_response = await openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": QUESTION},
                    {"role": "assistant", "content": None, "function_call": {"name": function_name, "arguments": message.function_call.arguments}},
                    {"role": "function", "name": function_name, "content": tool_response}
                ]
            )
            print(f"Final response: {final_response.choices[0].message.content}")
else:
    # function call이 없는 경우 직접 응답을 출력합니다
    print(f"Response: {message.content}")

ChatCompletion(id='chatcmpl-BGmgkSXHleQLlDTXdlBA5K83XVE2A', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "target": "apple"\n}', name='get_color'), tool_calls=None))], created=1743340106, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=104, total_tokens=120, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "target": "apple"\n}', name='get_color'), tool_calls=None)
ChatCompletion(id='chatcmpl-BGmgkSXHleQLlD

In [42]:
# 7. Make API call
response = await openai_client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": QUESTION1}
    ],
    functions=function_calls,
    function_call="auto"
)

# 8. Process the response
# 이 코드는 OpenAI API의 응답을 처리하는 부분입니다.
# OpenAI의 응답에서 첫 번째 메시지를 가져옵니다
message = response.choices[0].message
pprint.pprint(response)
pprint.pprint(message)

if message.function_call:
    # function call의 이름과 인자를 추출합니다
    function_name = message.function_call.name
    function_args = json.loads(message.function_call.arguments)
    
    # 요청된 도구를 찾아서 실행합니다
    for tool in tools:
        if tool.name == function_name:
            # 도구를 비동기로 실행하고 결과를 받습니다
            tool_response = await tool.ainvoke(function_args)
            print(f"Tool response: {function_name}")
            print(f"Tool response: {tool_response}")
            
            # 도구의 결과를 포함하여 최종 응답을 생성합니다
            final_response = await openai_client.chat.completions.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": QUESTION1},
                    {"role": "assistant", "content": None, "function_call": {"name": function_name, "arguments": message.function_call.arguments}},
                    {"role": "function", "name": function_name, "content": tool_response}
                ]
            )
            print(f"Final response: {final_response.choices[0].message.content}")
else:
    # function call이 없는 경우 직접 응답을 출력합니다
    print(f"Response: {message.content}")

ChatCompletion(id='chatcmpl-BGmgr5SfJitOYSkSyQZFC02d10oUi', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "target": "apple"\n}', name='get_name'), tool_calls=None))], created=1743340113, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=104, total_tokens=120, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=FunctionCall(arguments='{\n  "target": "apple"\n}', name='get_name'), tool_calls=None)
Tool response: get_name
Tool response: apple